In [64]:
from pathlib import Path
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from tqdm.auto import tqdm, trange

nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')

import string

import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

from sklearn.manifold import TSNE

import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure
% matplotlib inline

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Костя\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Костя\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Костя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
UsageError: Line magic function `%` not found.


In [48]:
path_to_books = Path('D:/Dev/GraduateProject/raw_books/')
stpwrds = stopwords.words('russian') + list(string.punctuation)


In [49]:
books = []
for x in path_to_books.rglob('*.txt'):
    with open(x, 'r', encoding='utf-8') as r:
        data = str(r.read())

    books.append(data.replace('\xa0', '').replace('\t', ''))


In [50]:
tk_books = []
for book in books:
    tk_books.append(sent_tokenize(book))

In [63]:
len(tk_books[2])

25762

In [53]:
data = {'author': [], 'name': [], 'text': []}
for book in tk_books:
    data['author'].append(book[0])
    data['name'].append(book[1])
    data['text'].append(book[2:])
df = pd.DataFrame(data)

In [57]:
df.to_csv('dataset.csv')

In [56]:
df['tag'] = [0, 1, 2]
# 0 - фантастика
# 1 - детектив
# 2 - фэнтези

In [78]:
df['text']

0    [Глава 1., Дом стоял на склоне холма на окpаин...
1    [1\nБыла половина октября, около одиннадцати у...
2    [ПРОЛОГ\n\n1., О ХОББИТАХ., Рассказ у нас пойд...
Name: text, dtype: object

In [76]:
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

In [85]:

for i, record in df.iterrows():
    print(record)

author                                        Адамс Дуглас.
name      Путеводитель по Галактике для путешествующих а...
text      [Глава 1., Дом стоял на склоне холма на окpаин...
tag                                                       0
Name: 0, dtype: object
author                                     Чэндлеp Раймонд.
name                                          Глубокий сон.
text      [1\nБыла половина октября, около одиннадцати у...
tag                                                       1
Name: 1, dtype: object
author                                      Дж.Р.Р.Толкиен.
name                                       ВЛАСТЕЛИН КОЛЕЦ.
text      [ПРОЛОГ\n\n1., О ХОББИТАХ., Рассказ у нас пойд...
tag                                                       2
Name: 2, dtype: object


In [92]:
texts = {'sentences': [], 'tag':[]}
for i, record in tqdm(df.iterrows()):
    for sent in record['text']:
        texts['sentences'].append(' '.join([remove_multiple_spaces(remove_numbers(remove_punctuation(w.lower()))) for w in word_tokenize(sent) if w not in stpwrds]))
        texts['tag'].append(record['tag'])

0it [00:00, ?it/s]

In [93]:
df_res = pd.DataFrame(texts)

In [94]:
df_res

,sentences,tag
0,глава,0
1,дом стоял склоне холма окpаине гоpодка отдельн...,0
2,окна выходили шиpокую pавнину западной англии,0
3,с стоpоны посмотpеть дом тpидцать лет это квад...,0
4,единственный кого дом чем то устpаивал аpтуp д...,0
...,...,...
36079,а сэм взял путь приречье подъехал круче закат ...,2
36080,прощальные бледно золотистые лучи озарили торб...,2
36081,роза встретила подвинула кресло камину усадила...,2
36082,он глубоко вздохнул,2


In [96]:
df_res.to_csv('dataset_sent.csv')